# GPT Bargaining

In [23]:
import openai
import re
import time

import numpy as np

from tqdm import tqdm
from agent import GPTAgent

In [2]:
openai.api_key = "sk-" 

# Initial Test

In [3]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

In [4]:
response

<OpenAIObject chat.completion id=chatcmpl-6pWyj9YUv3Ixyb1wzL8uQuAMyDBNh at 0x7ff629003f50> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The 2020 World Series was played in Arlington, Texas.",
        "role": "assistant"
      }
    }
  ],
  "created": 1677739517,
  "id": "chatcmpl-6pWyj9YUv3Ixyb1wzL8uQuAMyDBNh",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 15,
    "prompt_tokens": 56,
    "total_tokens": 71
  }
}

# Bargaining

In [24]:
seller_initial_instruction = open('initial_instructions/seller.txt').read()
seller = GPTAgent(initial_instruction=seller_initial_instruction)

In [25]:
seller.dialog_history

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': "Let's play a game. You are a balloon seller. You have a balloon to sell. You are bargaining with a buyer. The cost of your balloon is $10. Now you have a buyer asking for the price. Your start price is $20. Your goal is to sell it to a high price, as high as possible.\n\nYou should only respond to your buyer with one short, succinct sentence. Are your ready to play the game?"},
 {'role': 'assistant', 'content': "Yes, I'm ready to play the game!"}]

In [26]:
buyer_initial_instruction = open('initial_instructions/buyer.txt').read()
buyer = GPTAgent(initial_instruction=buyer_initial_instruction)

In [27]:
buyer.dialog_history

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': "Let's play a game. You are a balloon buyer. You want to buy a balloon. You are bargaining with a seller. You do not know the market price because you are entering a new country. Now you are asking a seller. Your goal is to buy it with a low price, as low as possible.\n\nYou should only respond to your buyer with one short, succinct sentence. \nNow ask a price. "},
 {'role': 'assistant', 'content': 'How much for the balloon?'}]

In [28]:
buyer_query = buyer.dialog_history[-1]['content']
for _ in tqdm(range(5)):
    seller_response = seller.call(buyer_query)
    buyer_query = buyer.call(seller_response)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.27it/s]


In [29]:
buyer.dialog_history

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': "Let's play a game. You are a balloon buyer. You want to buy a balloon. You are bargaining with a seller. You do not know the market price because you are entering a new country. Now you are asking a seller. Your goal is to buy it with a low price, as low as possible.\n\nYou should only respond to your buyer with one short, succinct sentence. \nNow ask a price. "},
 {'role': 'assistant', 'content': 'How much for the balloon?'},
 {'role': 'user',
  'content': "It's a high-quality balloon, exclusively priced at $20."},
 {'role': 'assistant', 'content': "I'll offer $10 for the balloon."},
 {'role': 'user',
  'content': 'I appreciate your offer, but this balloon is worth at least $17.'},
 {'role': 'assistant', 'content': 'How about $14 for the balloon?'},
 {'role': 'user',
  'content': "I can let it go for $16, but that's the lowest price I can offer for such a high-quality balloon."},
 {'role': 